In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

# For inline plots in Jupyter
%matplotlib inline
import pandas as pd
import matplotlib as plt
import glob
import os
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
from tqdm import tqdm
import codecs
import csv
import openpyxl

In [2]:
data_directory_weather = "/home/paulharford/college/project/project_data/processed/WEATHERED_warnings_2014-2023_cleaned_v3.csv"
data_directory_ihfd = "/home/paulharford/college/project/project_data/processed/ihfd_clean.csv"
data_directory_census = "/home/paulharford/college/project/project_data/processed/census_estimated_per_region_2014_2023.csv"
full_path_ihfd = os.path.abspath(data_directory_ihfd)
full_path_weather = os.path.abspath(data_directory_weather)
full_path_census = os.path.abspath(data_directory_census)

In [3]:
df_ihfd = pd.read_csv(full_path_ihfd)
df_weather = pd.read_csv(full_path_weather)
df_census = pd.read_csv(full_path_census)

In [4]:
df_weather.head()

,Issue Time,Valid From,Valid To,Warning Colour,Warning Element,Warning Text,Clare,Cork,Kerry,Limerick,...,warning_severity,weather_type,Duration_hours,region,date,triggered_counties,county_count,triggered_counties_str,has_multiple_events,warning_upgraded
0,2014-01-02 09:00:00+00:00,2014-01-02 17:00:00+00:00,2014-01-03 14:00:00+00:00,Orange,Wind,Becoming stormy this evening and tonight and c...,1,1,1,1,...,Orange,Wind warning,21.0,HSE West and North West,2014-01-02,"['Donegal', 'Sligo', 'Leitrim', 'Mayo', 'Galwa...",6,"Donegal, Sligo, Leitrim, Mayo, Galway, Roscommon",0,1
1,2014-01-02 09:00:00+00:00,2014-01-02 17:00:00+00:00,2014-01-03 14:00:00+00:00,Orange,Wind,Becoming stormy this evening and tonight and c...,1,1,1,1,...,Orange,Wind warning,21.0,HSE West and North West,2014-01-03 00:00:00,"['Donegal', 'Sligo', 'Leitrim', 'Mayo', 'Galwa...",6,"Donegal, Sligo, Leitrim, Mayo, Galway, Roscommon",0,1
2,2014-01-02 09:00:00+00:00,2014-01-02 17:00:00+00:00,2014-01-03 14:00:00+00:00,Orange,Wind,Becoming stormy this evening and tonight and c...,1,1,1,1,...,Orange,Wind warning,21.0,HSE Mid West,2014-01-02,"['Clare', 'Limerick']",2,"Clare, Limerick",1,1
3,2014-01-02 09:00:00+00:00,2014-01-02 17:00:00+00:00,2014-01-03 14:00:00+00:00,Orange,Wind,Becoming stormy this evening and tonight and c...,1,1,1,1,...,Orange,Wind warning,21.0,HSE Mid West,2014-01-03 00:00:00,"['Clare', 'Limerick']",2,"Clare, Limerick",1,1
4,2014-01-02 09:00:00+00:00,2014-01-02 17:00:00+00:00,2014-01-03 14:00:00+00:00,Orange,Wind,Becoming stormy this evening and tonight and c...,1,1,1,1,...,Orange,Wind warning,21.0,HSE South West,2014-01-02,"['Cork', 'Kerry']",2,"Cork, Kerry",0,1


In [5]:
df_weather.rename(columns={"region": "Region"}, inplace=True)

In [6]:
df_weather["Valid From"] = pd.to_datetime(df_weather["Valid From"])
df_weather["date"] = df_weather["Valid From"].dt.date
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather['Year'] = df_weather['date'].dt.year

In [7]:
severity_mapping = {
    'Yellow': 1,
    'Orange': 2,
    'Red': 3
}

df_weather['warning_severity_numeric'] = df_weather['warning_severity'].map(severity_mapping)

In [8]:
def aggregate_weather_data(df_weather):
    """
    Aggregate weather data by Region, date, and weather_type to maintain 
    separate rows for different weather events on the same day.
    
    Args:
        df_weather: DataFrame containing weather data with Region, date, and weather_type columns
        
    Returns:
        Aggregated DataFrame with one row per unique Region-date-weather_type combination
    """
    # First step: Group by Region, date, and weather_type
    # This ensures separate events remain as separate rows
    weather_agg = df_weather.groupby(
        ['Region', 'date', 'weather_type', 'warning_severity'], 
        as_index=False
    ).agg({
        'county_count': 'max',
        'warning_severity_numeric': 'max'
    })
    
    # Add the weather flag column (set to 1 if there is any event)
    weather_agg['weather_event'] = 1
    
    return weather_agg

In [9]:
weather_agg = aggregate_weather_data(df_weather)

In [10]:
weather_agg['Year'] = weather_agg['date'].dt.year

In [11]:
weather_agg.head(15)

,Region,date,weather_type,warning_severity,county_count,warning_severity_numeric,weather_event,Year
0,HSE Dublin and Midlands,2014-01-02,Wind warning,Yellow,7,1.0,1,2014
1,HSE Dublin and Midlands,2014-01-03,Wind warning,Yellow,7,1.0,1,2014
2,HSE Dublin and Midlands,2014-01-04,Snow_Ice warning,Yellow,7,1.0,1,2014
3,HSE Dublin and Midlands,2014-01-05,Wind warning,Yellow,7,1.0,1,2014
4,HSE Dublin and Midlands,2014-01-06,Wind warning,Orange,7,2.0,1,2014
5,HSE Dublin and Midlands,2014-01-06,Wind warning,Yellow,7,1.0,1,2014
6,HSE Dublin and Midlands,2014-01-12,Rainfall warning,Yellow,4,1.0,1,2014
7,HSE Dublin and Midlands,2014-01-12,Wind warning,Yellow,7,1.0,1,2014
8,HSE Dublin and Midlands,2014-01-16,Fog warning,Orange,7,2.0,1,2014
9,HSE Dublin and Midlands,2014-01-25,Wind warning,Yellow,7,1.0,1,2014


In [12]:
import pandas as pd
import plotly.express as px
from shiny import App, ui, reactive, render
from shiny import App, ui, reactive
from shinywidgets import output_widget, render_widget  #

In [13]:
# 1) Define a user interface
app_ui = ui.page_fluid(
    ui.h2("Adverse Weather Events in Ireland"),
    ui.layout_sidebar(
        ui.sidebar(
            ui.input_select(
                "region_select",
                "Choose a region:",
                choices=sorted(weather_agg["Region"].unique()),
                selected=sorted(weather_agg["Region"].unique())[0]
            ),
            ui.input_slider(
                "year_range",
                "Select Year Range:",
                min=weather_agg["Year"].min(),
                max=weather_agg["Year"].max(),
                value=(weather_agg["Year"].min(), weather_agg["Year"].max()),
                step=1
            ),
            ui.input_checkbox_group(
                "weather_type_select",
                "Select Weather Type:",
                choices=sorted(weather_agg["weather_type"].unique()),
                selected=sorted(weather_agg["weather_type"].unique())
            ),
            ui.input_checkbox_group(
                "warning_severity_select",
                "Select Weather Severity:",
                choices=sorted(weather_agg["warning_severity"].unique()),
                selected=sorted(weather_agg["warning_severity"].unique())
            ),
            ui.input_radio_buttons(
                "viz_type",
                "Visualization Type:",
                choices=["Total Counts", "By Weather Type", "By Weather Severity"],
                selected="Total Counts"
            )
        ),
        output_widget("weather_plot")
    )
)

# 2) Define the server setup
def server(input, output, session):
    @reactive.Calc
    def filtered_data():
        selected_region = input.region_select()
        year_min, year_max = input.year_range()
        selected_type = input.weather_type_select()
        selected_severity = input.warning_severity_select()
        
        # Filter by all selected criteria
        dff = weather_agg[
            (weather_agg["Region"] == selected_region) &
            (weather_agg["Year"] >= year_min) &
            (weather_agg["Year"] <= year_max) &
            (weather_agg["weather_type"].isin(selected_type)) &
            (weather_agg["warning_severity"].isin(selected_severity))
        ]
        
        return dff
    
    @render_widget
    def weather_plot():
        dff = filtered_data()
        viz_type = input.viz_type()
        
        if len(dff) == 0:
            # Return a blank figure with a message
            fig = px.line(title="No data available for selection.")
            fig.update_layout(
                xaxis={"visible": False},
                yaxis={"visible": False},
                annotations=[
                    {
                        "text": "No data available for the current selection.",
                        "xref": "paper",
                        "yref": "paper",
                        "showarrow": False,
                        "font": {"size": 18}
                    }
                ]
            )
            return fig
        
        if viz_type == "Total Counts":
            # Count events by year
            event_counts = dff.groupby("Year").size().reset_index(name="Event_Count")
            
            fig = px.line(
                event_counts,
                x="Year",
                y="Event_Count",
                title=f"Adverse Weather Events in {input.region_select()}",
                markers=True
            )
            
        elif viz_type == "By Weather Type":
            # Group by Year and weather type
            type_counts = dff.groupby(["Year", "weather_type"]).size().reset_index(name="Event_Count")
            
            fig = px.line(
                type_counts,
                x="Year",
                y="Event_Count",
                color="weather_type",
                title=f"Weather Events by Type in {input.region_select()}",
                markers=True
            )
            
        elif viz_type == "By Weather Severity":
            # Group by Year and Severity
            severity_counts = dff.groupby(["Year", "warning_severity"]).size().reset_index(name="Event_Count")
            
            # Create a custom color map for severity levels
            color_map = {"Red": "red", "Orange": "orange", "Yellow": "gold", "Unknown": "gray"}
            
            fig = px.line(
                severity_counts,
                x="Year",
                y="Event_Count",
                color="warning_severity",
                color_discrete_map=color_map,
                title=f"Weather Events by Severity in {input.region_select()}",
                markers=True
            )
        
        # Improve the figure layout
        fig.update_layout(
            xaxis_title="Year",
            yaxis_title="Number of Adverse Weather Events",
            legend_title_text="",
            template="plotly_white"
        )
            
        return fig

# 3) Create App object
app = App(app_ui, server)

In [14]:
import shiny
print(shiny.__version__)

1.2.1


In [15]:
import nest_asyncio
nest_asyncio.apply()

# Now you can run the app
app.run()


INFO:     Started server process [652958]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52344 - "GET / HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 52452) - "WebSocket /websocket/" [accepted]
INFO:     connection open
INFO:     Shutting down
INFO:     connection closed
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [652958]
